# Medium Math Calculator in Haskell

## Imports

In [223]:
import Data.Char

## Types, DataTypes & Constants

In [224]:
data MathValue =
      Empty
    | Value Float
    | RawExpr String
    | Operator Char
    deriving (Show)

In [225]:
test_string = "1 + 2 * (4 - 5) / 6"

In [226]:
operators :: [Char]
operators = "%/*-+"

## Parsing & String Utils

In [227]:
-- purge input string
removeWhiteSpaces :: String -> String
removeWhiteSpaces = filter (not . isSpace)

-- same char class
sameCharClass :: Char -> Char -> Bool
sameCharClass c1 c2
    | c1 `elem` operators || c2 `elem` operators = False
    | isDigit c1 && isDigit c2                   = True
    | c1 == '(' || c2 == ')'                     = True
    | otherwise                                  = False

-- get the next mathematical value in a string
-- e.g. in "1+2", the restult would be ("1", "+2")
getNextMathValueString :: String -> Int -> (String, String)
getNextMathValueString [] _ = ("", "")
getNextMathValueString [c] _ = ([c], "")
getNextMathValueString (c1 : c2 : s) n
    | c1 == ')' = if n == 1 then ([c1], c2 : s) -- closing parenthesis expr
        else let (s2, rs) = getNextMathValueString (c2 : s) (n - 1) in (c1 : s2, rs) -- closing nested parenthesis
    | c1 == '(' = if n == 0 then let (s2, rs) = getNextMathValueString (c2 : s) 1 in (c1 : s2, rs)
        else let (s2, rs) = getNextMathValueString (c2 : s) (n + 1) in (s2, rs)
    | c2 == '(' = if n == 0 then ([c1], c2: s)
        else let (s2, rs) = getNextMathValueString (c2 : s) n in (c1 : c2 : s2, rs) -- next to an opening parenthesis
    | n /= 0 = let (s2, rs) = getNextMathValueString (c2 : s) n in (c1 : s2, rs) -- inside nested parenthesis
    | sameCharClass c1 c2 = let (s2, rs) = getNextMathValueString (c2 : s) n in (c1 : s2, rs)
    | otherwise = ([c1], c2 : s)

In [228]:
getNextMathValueString "(12*4)+2+3" 0
getNextMathValueString "(12)+2+3" 0
getNextMathValueString "(1)+2+3" 0
getNextMathValueString "(1*(2/3))+2+3" 0
getNextMathValueString "(1*(2/3)-4)+2+3" 0
getNextMathValueString "" 0

("(12*4)","+2+3")

("(12)","+2+3")

("(1)","+2+3")

("(1*(2/3))","+2+3")

("(1*(2/3)-4)","+2+3")

("","")

In [229]:
stringToMathValue :: String -> MathValue
stringToMathValue s
    | s == ""                                  = Empty
    | head s == '('                            = RawExpr s
    | length s == 1 && head s `elem` operators = Operator $ head s
    | otherwise                                = Value $ read s

In [230]:
stringToMathValue "12"
stringToMathValue "(12)"
stringToMathValue ""
stringToMathValue "+"

Value 12.0

RawExpr "(12)"

Empty

Operator '+'

In [231]:
parseString :: String -> [MathValue]
parseString s =
    let (value_str,rest) = getNextMathValueString s 0
    in let value = stringToMathValue value_str
    in if null rest then [value] else value : parseString rest

In [232]:
parseString "12+4*(0.1-(3/4)+4)/6"

[Value 12.0,Operator '+',Value 4.0,Operator '*',RawExpr "(0.1-(3/4)+4)",Operator '/',Value 6.0]

## Evaluation

In [233]:
evalOperator :: Char -> Float -> Float -> Float
evalOperator op v1 v2 =
    case op of
        '+' -> v1 + v2
        '-' -> v1 - v2
        '*' -> v1 * v2
        '/' -> v1 / v2
        _   -> error ("Unknown operator " ++ [op])

In [234]:
evalOperator '+' 1 2
evalOperator '&' 1 2

3.0

: 

In [235]:
evalMathValue :: MathValue -> Float
evalMathValueList :: [MathValue] -> Float
evalMathValueListSingleOperator :: Char -> [MathValue] -> [MathValue]

evalMathValue Empty = error "Cannot evaluate an empty expression"
evalMathValue (Value v) = v
evalMathValue (RawExpr r) = evalMathValueList $ parseString $ tail $ init r -- remove the parentheses b4 parsing
evalMathValue (Operator op) = error "Cannot evaluate an operator"

evalMathValueListSingleOperator _ [] = error "Cannot evaluate an empty expression list"
evalMathValueListSingleOperator _ [v] = [v]
evalMathValueListSingleOperator _ [_, _] = error "Cannot evaluate expression with two operands (must be odd)"
evalMathValueListSingleOperator op (a:(Operator other_op):b:rest) =
    if op == other_op then
        let value = evalOperator op (evalMathValue a) (evalMathValue b)
        in evalMathValueListSingleOperator op (Value value:rest)
    else
        a : Operator other_op : evalMathValueListSingleOperator op (b : rest)

evalMathValueList mvl = evalMathValue $ head $ foldl (flip evalMathValueListSingleOperator) mvl operators

In [236]:
evalMathValueList $ parseString "(12+4*(3/4)/6)"
12+4*(3/4)/6

12.5

12.5

## Wrapping into a final function

In [239]:
calculate :: String -> Float
calculate s = evalMathValueList . parseString $ removeWhiteSpaces s

In [240]:
calculate "12 * 4 / (5 - 1) * 5"
12 * 4 / (5 - 1) * 5

60.0

60.0